# Confirm that noun banks are accessible by ipykernal.
## Shared Pandas dfs are transported as pickle files.
## Read using the %store -r '...' 'magic method'

In [7]:
# %store many_capitals
# %store many_us_presidents
# %store many_english_words
# %store all_usa_states
# %store all_world_countries
# %store all_elements
# %store many_car_manufacturers
# %store many_cosmetic_items
# %store many_fruits
# %store many_luxury_brands
# %store many_olympic_sports
# %store many_greek_gods

%store -r many_capitals
%store -r many_us_presidents
%store -r many_english_words
%store -r all_usa_states
%store -r all_world_countries
%store -r all_elements
%store -r many_car_manufacturers
%store -r many_cosmetic_items
%store -r many_fruits
%store -r many_luxury_brands
%store -r many_olympic_sports
%store -r many_greek_gods

## Delete old database

In [8]:
import os
#delete old database
os.remove('NounBankSQlite.db')


FileNotFoundError: [Errno 2] No such file or directory: 'NounBankSQlite.db'


# Creating an engine will store a file "NounBankSQlite.db" at SQlite-Databases/NounBankSQlite.db
#### 

In [12]:
import pandas as pd
from pandas.io import sql
from sqlalchemy import create_engine

DATABASE_LOCATION = "sqlite:///NounBankSQlite.db"

    
def post_to_sqlite():
    engine = create_engine(DATABASE_LOCATION)
    many_capitals.to_sql("many_capitals", engine, index=False, if_exists='replace')
    many_us_presidents.to_sql("many_us_presidents", engine, index=False, if_exists='replace') # make sure table name and col name are always the same
    many_english_words.to_sql("many_english_words", engine, index=False, if_exists='replace')
    all_usa_states.to_sql("all_usa_states", engine, index=False, if_exists='replace')
    all_world_countries.to_sql("all_world_countries", engine, index=False, if_exists='replace')
    all_elements.to_sql("all_elements", engine, index=False, if_exists='replace')
    many_car_manufacturers.to_sql("many_car_manufacturers", engine, index=False, if_exists='replace')
    many_cosmetic_items.to_sql("many_cosmetic_items", engine, index=False, if_exists='replace')
    many_fruits.to_sql("many_fruits", engine, index=False, if_exists='replace')
    many_luxury_brands.to_sql("many_luxury_brands", engine, index=False, if_exists='replace')
    many_olympic_sports.to_sql("many_olympic_sports", engine, index=False, if_exists='replace')
    many_greek_gods.to_sql("many_greek_gods", engine, index=False, if_exists='replace')
    
    
    print("Data posted to SQLlite database!")
    
    engine.dispose()

    # close database connection
    

post_to_sqlite()

Data posted to SQLlite database!


## Confirm that future updates to the Noun Bank do not raise any assertion errors

In [10]:
import unittest as ut

class TestPostToSqlite(ut.TestCase):
    def __init__(self, *args, **kwargs):
        self.engine = create_engine(DATABASE_LOCATION) # create an engine to connect to the database for testing
        super(TestPostToSqlite, self).__init__(*args, **kwargs) # call the parent class's constructor

    def test_connecting_to_sqlite(self):
        self.assertEqual(self.engine.name, 'sqlite') # check that the engine is connected to a sqlite database
        
    def test_states_start_with_new(self):
        df = pd.read_sql('SELECT all_usa_states FROM all_usa_states WHERE all_usa_states LIKE "New%"', self.engine) # select all states that start with "New"
        self.assertEqual(df.values.tolist(), [['New Hampshire'], ['New Jersey'], ['New Mexico'], ['New York']]) # check that the correct states are returned
        
    def test_longest_len_greek_god(self):
        df = pd.read_sql('SELECT MAX(LENGTH(many_greek_gods)), many_greek_gods FROM many_greek_gods', self.engine)
        self.assertEqual(df['many_greek_gods'][0], 'hephaestus' ) # check that the greek god with the longest name is Hephaestus
        self.assertEqual(df['MAX(LENGTH(many_greek_gods))'][0], 10) # check that the length of the longest name is 11

    def test_element_with_largest_atomic_number(self):
        df = pd.read_sql('SELECT all_elements, AtomicNumber FROM all_elements WHERE AtomicNumber = (SELECT MAX(AtomicNumber) FROM all_elements)', self.engine)
        self.assertEqual(df['all_elements'][0], 'Oganesson') # check that the element with the largest atomic number is Oganesson
        self.assertEqual(df['AtomicNumber'][0], 118) # check that the atomic number of Oganesson is 118


#run the tests

if __name__ == '__main__':
    ut.main(argv=[''], verbosity=4, exit=False)

test_connecting_to_sqlite (__main__.TestPostToSqlite) ... ok
test_element_with_largest_atomic_number (__main__.TestPostToSqlite) ... ERROR
test_longest_len_greek_god (__main__.TestPostToSqlite) ... ERROR
test_states_start_with_new (__main__.TestPostToSqlite) ... ERROR

ERROR: test_element_with_largest_atomic_number (__main__.TestPostToSqlite)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/sqlalchemy/engine/base.py", line 1799, in _execute_context
    self.dialect.do_execute(
  File "/usr/local/lib/python3.10/site-packages/sqlalchemy/engine/default.py", line 717, in do_execute
    cursor.execute(statement, parameters)
sqlite3.OperationalError: no such table: all_elements

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/var/folders/nx/n1nd7k552014npzf4p5zpxh00000gn/T/ipykernel_34738/1199976119.py", line 21, in test_el

In [18]:
import numpy as np


class Question:
    def __init__(self) -> None:
        self.engine = create_engine(DATABASE_LOCATION)
        self.available_tables = pd.read_sql('SELECT name FROM sqlite_master WHERE type="table"', self.engine)
        # print(self.available_tables)
        self.chosen_table = np.random.choice(self.available_tables['name'])

        if self.chosen_table == 'all_usa_states':
            self.NOUN = "A State in the USA"
        elif self.chosen_table == 'all_world_countries':
            self.NOUN = "A Country in the World"
        elif self.chosen_table == 'all_elements':
            self.NOUN = "An Element"
        elif self.chosen_table == 'many_animals':
            self.NOUN = "An Animal"
        elif self.chosen_table == 'many_greek_gods':
            self.NOUN = "A Greek God"
        elif self.chosen_table == 'many_roman_gods':
            self.NOUN = "A Roman God"
        elif self.chosen_table == 'many_cosmetic_items':
            self.NOUN = "A Cosmetic Item"
        elif self.chosen_table == 'many_office_supplies':
            self.NOUN = "An Office Supply"
        elif self.chosen_table == 'many_olympic_sports':
            self.NOUN = "An Olympic Sport"
        elif self.chosen_table == 'many_nfl_teams':
            self.NOUN = "An NFL Team"
        elif self.chosen_table == 'many_marvel':
            self.NOUN = "A Marvel Character"
        # %store -r many_mythical_creatures
        # %store -r many_weather_conditions
        # %store -r many_vegetables
        # %store -r many_professions
        # %store -r many_pets
        # %store -r many_landmarks
        # %store -r many_luxury_brands
        # %store -r many_holidays
        # %store -r many_gemstones
        # %store -r many_fruits
        # %store -r many_flowers_plants
        # %store -r many_empires
        # %store -r many_youtubers
        elif self.chosen_table == 'many_mythical_creatures':
            self.NOUN = "A Mythical Creature"
        elif self.chosen_table == 'many_weather_conditions':
            self.NOUN = "A Weather Condition"
        elif self.chosen_table == 'many_vegetables':
            self.NOUN = "A Vegetable"
        elif self.chosen_table == 'many_professions':
            self.NOUN = "A Profession"
        elif self.chosen_table == 'many_pets':
            self.NOUN = "A Pet"
        elif self.chosen_table == 'many_landmarks':
            self.NOUN = "A Landmark"
        elif self.chosen_table == 'many_luxury_brands':
            self.NOUN = "A Luxury Brand"
        elif self.chosen_table == 'many_holidays':
            self.NOUN = "A Holiday"
        elif self.chosen_table == 'many_gemstones':
            self.NOUN = "A Gemstone"
        elif self.chosen_table == 'many_fruits':
            self.NOUN = "A Fruit"
        elif self.chosen_table == 'many_flowers_plants':
            self.NOUN = "A Flower or Plant"
        elif self.chosen_table == 'many_empires':
            self.NOUN = "An Empire"
        elif self.chosen_table == 'many_youtubers':
            self.NOUN = "A Youtuber"

        self.question_filters = ["Begins With The Letter", "Ends With The Letter", "Any"]

    def get_question(self):
        # print(self.options)
        self.chosen_filter = np.random.choice(self.question_filters)
        # make a list of all the first letters of the nouns in the chosen table
        if self.chosen_filter == "Any":
            pass

        if self.chosen_filter == "Begins With The Letter":
            # make a set called first letters that contains all the first letters of the nouns in the chosen table
            first_letters = pd.read_sql('SELECT SUBSTR(' + self.chosen_table + ', 1, 1) FROM ' + self.chosen_table, self.engine)
            # make one dimensional array of first letters
            first_letters = first_letters.values.tolist()
            # make first letters into a list rather than a list of lists
            first_letters = [item for sublist in first_letters for item in sublist]
            self.random_letter = np.random.choice(first_letters)
            # print(first_letters)
            
        if self.chosen_filter == "Ends With The Letter":
            # make a set called last letters that contains all the last letters of the nouns in the chosen table
            last_letters = pd.read_sql('SELECT SUBSTR(' + self.chosen_table + ', -1, 1) FROM ' + self.chosen_table, self.engine)
            # make one dimensional array of first letters
            last_letters = last_letters.values.tolist()
            # make first letters into a list rather than a list of lists
            last_letters = [item for sublist in last_letters for item in sublist]
            self.random_letter = np.random.choice(last_letters)
            # print(last_letters)
        
        if self.chosen_filter == "Begins With The Letter" or self.chosen_filter == "Ends With The Letter":
            return "Name " + self.NOUN + " That " + self.chosen_filter + " " + self.random_letter.capitalize() + "."
        
        if self.chosen_filter == "Any":
            return "Name " + self.NOUN + "."

    def check_answer(self, potential_answer):
        if self.chosen_filter == "Any":
            # check if potential answer is in the chosen table in the database
            all_nouns_in_table = pd.read_sql('SELECT ' + self.chosen_table + ' FROM ' + self.chosen_table, self.engine)
            # print(all_nouns_in_table.values.tolist())
            all_nouns_in_table = [item for sublist in all_nouns_in_table.values.tolist() for item in sublist]
            # make all nouns in table lowercase
            all_nouns_in_table = [x.lower() for x in all_nouns_in_table]
            # check if potential answer is in the list of all nouns in the table
            if potential_answer.lower() in all_nouns_in_table:
                print("Correct!")
                print(potential_answer.lower().capitalize())
            else:
                print("Incorrect!")
                print(potential_answer.lower().capitalize() + " is not a " + self.NOUN.lower() + ".")
                
        if self.chosen_filter == "Begins With The Letter":
            if potential_answer[0].lower() == self.random_letter.lower():
                # check if potential answer is in the chosen table in the database
                all_nouns_in_table = pd.read_sql('SELECT ' + self.chosen_table + ' FROM ' + self.chosen_table, self.engine)
                # print(all_nouns_in_table.values.tolist())
                all_nouns_in_table = [item for sublist in all_nouns_in_table.values.tolist() for item in sublist]
                # make all nouns in table lowercase
                all_nouns_in_table = [x.lower() for x in all_nouns_in_table]
                # check if potential answer is in the list of all nouns in the table
                if potential_answer.lower() in all_nouns_in_table:
                    print("Correct!")
                    print(potential_answer.lower().capitalize() + " is a " + self.NOUN + " that begins with the letter " + self.random_letter + ".")
                else:
                    print("That is not a " + self.NOUN + "!")
            else:
                print("That does not begin with the letter " + self.random_letter + "!")
                
        if self.chosen_filter == "Ends With The Letter":
            if potential_answer[-1].lower() == self.random_letter.lower():
                # check if potential answer is in the chosen table in the database
                all_nouns_in_table = pd.read_sql('SELECT ' + self.chosen_table + ' FROM ' + self.chosen_table, self.engine)
                # print(all_nouns_in_table.values.tolist())
                all_nouns_in_table = [item for sublist in all_nouns_in_table.values.tolist() for item in sublist]
                # make all nouns in table lowercase
                all_nouns_in_table = [x.lower() for x in all_nouns_in_table]
                # check if potential answer is in the list of all nouns in the table
                if potential_answer.lower() in all_nouns_in_table:
                    print("CORRECT!")
                    print(potential_answer.lower().capitalize() + " is a " + self.NOUN + " that ends with the letter " + self.random_letter + ".")
                else:
                    print("That is not a " + self.NOUN + "!")
            else:
                print("That does not end with the letter " + self.random_letter + "!")

        

In [11]:
for round in range(10):
    q = Question()
    print(q.get_question())
    a = input("Answer: ")
    q.check_answer(a)
    print("\n")


NameError: name 'Question' is not defined